In [13]:
experiments_dev = ["WFV1_unet", "WF1_unet_all", "WF1_unet_bgr", "WF1_unet_rgbiswir", "WF1_unet_rgbi",
                   "WF1_hrnet_rgbbs32", "WF1_hrnet_allbs32", "WF2_unet", "WF2_hrnetsmall_rgb","WFV1_scnn20", "WF1_scnn_bgr",
                   "WF1_simplecnn_rgbi", "WF1_simplecnn_rgbiswir"]

In [7]:
from ml4floods.data import utils
from ml4floods.models.utils import metrics
import pandas as pd

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/creds.json"

In [8]:
def get_metrics_percode(mets):
    iou_per_code = pd.DataFrame(metrics.group_confusion(mets["confusions"],mets["cems_code"], metrics.calculate_iou,
                                                        label_names=[f"IoU_{l}"for l in ["land", "water", "cloud"]]))

    recall_per_code = pd.DataFrame(metrics.group_confusion(mets["confusions"],mets["cems_code"], 
                                                           metrics.calculate_recall,
                                                           label_names=[f"Recall_{l}"for l in ["land", "water", "cloud"]]))

    join_data_per_code = pd.merge(recall_per_code,iou_per_code,on="code")
    join_data_per_code = join_data_per_code.set_index("code")
    join_data_per_code = join_data_per_code*100
    return join_data_per_code
    # print(f"Mean values across flood events: {join_data_per_code.mean(axis=0).to_dict()}")

In [16]:
import itertools

prod_dev = "0_DEV"
dl_name = "test"

experiments_stats = []

for e in experiments_dev:
    # Obtain band configuration and model
    config = utils.read_json_from_gcp(f"gs://ml4cc_data_lake/{prod_dev}/2_Mart/2_MLModelMart/{e}/config.json")
    channels = config["model_params"]["hyperparameters"]["channel_configuration"]
    architecture = config["model_params"]["hyperparameters"]["model_type"]
    
    for dl_name in ["val", "test"]:
        data = utils.read_json_from_gcp(f"gs://ml4cc_data_lake/{prod_dev}/2_Mart/2_MLModelMart/{e}/{dl_name}.json")

        # compute metrics per flood
        metrics_per_code = get_metrics_percode(data)
        mean_metrics_code = metrics_per_code.mean(axis=0).to_dict()


        experiments_stats.append({
            "global_recall" : data["recall"]["water"],
            "global_iou" : data["iou"]["water"],
            "mean_recall_perflood" : mean_metrics_code["Recall_water"],
            "mean_iou_perflood" : mean_metrics_code["IoU_water"],
            "experiment": e,
            "channels": channels,
            "architecture": architecture,
            "subset": dl_name
        })

experiments_stats = pd.DataFrame(experiments_stats)
experiments_stats

,global_recall,global_iou,mean_recall_perflood,mean_iou_perflood,experiment,channels,architecture,subset
0,0.968581,0.702739,94.291257,57.659955,WFV1_unet,all,unet,val
1,0.945968,0.729611,91.764970,74.148756,WFV1_unet,all,unet,test
2,0.970897,0.661126,95.619078,52.412905,WF1_unet_all,all,unet,val
3,0.958935,0.713364,92.150262,68.785204,WF1_unet_all,all,unet,test
4,0.945311,0.539748,85.779459,37.647084,WF1_unet_bgr,bgr,unet,val
5,0.622788,0.388266,75.951768,39.045404,WF1_unet_bgr,bgr,unet,test
6,0.964606,0.710745,94.176290,56.292781,WF1_unet_rgbiswir,bgriswir,unet,val
7,0.941331,0.648547,90.338509,70.039992,WF1_unet_rgbiswir,bgriswir,unet,test
8,0.966831,0.689834,94.585864,53.648376,WF1_unet_rgbi,bgri,unet,val
9,0.909095,0.639452,88.550093,62.933332,WF1_unet_rgbi,bgri,unet,test
